In [1]:
import json
import pika
import uuid
import threading 
import datetime
import time
import logging
import random
import requests
import os
from enum import Enum
from flask import Flask, request, jsonify
from flask_api import status
from pymessenger import Element, Button
from requests_toolbelt import MultipartEncoder
from pymessenger import utils
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
class NotificationType(Enum):
    regular = "REGULAR"
    silent_push = "SILENT_PUSH"
    no_push = "NO_PUSH"

In [3]:
def verify_fb_token(token_sent,exito):
    url = "http://127.0.0.1:8080/access_token"    
    payload = "{\"access_token\": \"" + token_sent +"\"}"
    headers = {
        'content-type': "application/json",
        'cache-control': "no-cache",
        'postman-token': "64225f1a-4ded-800c-14f2-74ff3a3d45b9"
        }
    response = requests.request("POST", url, data=payload, headers=headers)   
    access_tokens=response.json()["access_token"]    
    if token_sent in access_tokens:
        return exito
    else:
        return 'Verificacion de token invalida'

In [4]:
def enviarNucleo(recipient_id, id_face, mensaje,hilo):
    global response
    global corr_id        
    cont=0
    content=''
    type_resp=''    
    if "postback" in mensaje[0]:
        content=mensaje[0]["postback"]["payload"]
        type_resp="postback"
    elif "message" in mensaje[0]:
        content=mensaje[0]["message"]["text"]
        type_resp="message"    
    entidad={}
    estado = "none"
    tipo="none"
    opciones="none"     
    n =  {"payload": {"robot":" ","id_face":" ","recipient_id":" ","access_token":" ","content":" ","type_resp":" ","state":" ", "blocktype":" ", "opciones":" ", "entitites":" "}}
    data=json.dumps(n)
    decoded=json.loads(data)
    decoded["payload"]["robot"]="none"
    decoded["payload"]["id_face"]=id_face
    decoded["payload"]["access_token"]="none"
    decoded["payload"]["recipient_id"]=recipient_id
    decoded["payload"]["content"]=content
    decoded["payload"]["type_resp"]=type_resp
    decoded["payload"]["state"]=estado
    decoded["payload"]["blocktype"]=tipo
    decoded["payload"]["opciones"]=opciones
    decoded["payload"]["entities"]= entidad
    data_string=json.dumps(decoded)    
    hilo.corr_id = str(uuid.uuid4())
    hilo.channel.basic_consume(on_message_callback=hilo.on_response, auto_ack=True, queue=hilo.queue_name) 
    try:
        hilo.channel.basic_publish(exchange='',
                              routing_key='Colaprueba22',
                              properties=pika.BasicProperties(
                                        reply_to = hilo.queue_name,
                                        correlation_id = hilo.corr_id,
                                        ),
                                body=str(data_string))
        hilo.start()        
        print('\n_________________________________________________________________________________________________________________________')
        print('Mensaje enviado al Nucleo Logico-> ',data_string)        
    except Exception as error:
        print("mensaje no enviado...",str(error))

In [5]:
def send_message(payload,access_token):
    api_version = 6.0
    graph_url = 'https://graph.facebook.com/v{0}'.format(api_version)
    request_endpoint = '{0}/me/messages'.format(graph_url)
    auth = {'access_token': access_token}
    print('\n_________________________________________________________________________________________________________________________')
    print('Mensaje enviado a la API de  Facebook')
    print('request_endpoint-> ',request_endpoint)
    print('ruta->',auth)
    print('payload-> ',payload)        
    response = requests.post(
        request_endpoint,
        params=auth,
        json=payload
    )
    result = response.json()
    return result

In [6]:
def text_message(recipient_id, response,access_token):
    payload={'message': {'text': response}, 'recipient': {'id': recipient_id}, 'notification_type': 'REGULAR'}
    print("enviando text message",payload)
    respuesta=send_message(payload,access_token)
    print("respuesta:",respuesta)
    return "success"

In [7]:
def button_message(recipient_id,content,options,access_token):
    buttons = []
    cont=0
    for opc in options:
        button = Button(title=opc, type='postback', payload=opc)
        buttons.append(button)
        cont=cont+1
        if cont>2:
            break    
    payload={'message': {'attachment': {'type': 'template', 'payload': {'template_type': 'button', 'text': content, 'buttons': buttons}}}, 'recipient': {'id': recipient_id}, 'notification_type': 'REGULAR'}
    
    result=send_message(payload,access_token)
    assert type(result) is dict
    assert result.get('message_id') is not None
    assert result.get('recipient_id') is not None

In [8]:
def slide_message(recipient_id,content,access_token):
    buttons =[]
    elements=[]
    element={'title':'', 'image_url':'', 'subtitle':'', 'buttons':[]}
    cont=0
    
    for data in content: 
        buttons =[]
        for btn in data['buttons']:
            if btn['typeButton']=='web_url':
                button = Button(title=btn['titleButton'], type='web_url', url=btn['urlButton'])                
            elif btn['typeButton']=='postback':
                button = Button(title=btn['titleButton'], type='postback', payload=btn['payloadButton'])
            buttons.append(button)
            cont=cont+1
            if cont>2:
                break
        element={'title':data['title'], 'image_url':data['image_url'], 'subtitle':data['subtitle'], 'buttons':buttons}
        elements.append(element)   
        
    payload={"recipient": {"id":recipient_id}, "message":{"attachment": {"type":"template", "payload": {"template_type":"generic", "elements":elements}}}, 'notification_type': 'REGULAR'}
    
    result=send_message(payload,access_token)
    assert type(result) is dict
    assert result.get('message_id') is not None
    assert result.get('recipient_id') is not None

In [9]:
def ticket_message(recipient_id,content,access_token):
    payload={"recipient":{"id":recipient_id},"message":{"attachment":{"type":"template","payload":content}}}    
    result=send_message(payload,access_token)
    assert type(result) is dict
    assert result.get('message_id') is not None
    assert result.get('recipient_id') is not None

In [10]:
def conv(msj):
    ans=' '
    contenido=msj["Respuesta"]["content"]
    estado=msj["Respuesta"]["payload"]["state"]
    entidad= msj["Respuesta"]["payload"]["entities"]
    tipo= msj["Respuesta"]["blockType"]
    recipient_id=msj['Respuesta']['payload']['recipient_id']
    access_token=msj['Respuesta']['payload']['access_token']    
    
    if(tipo=="quickReply" or tipo=="quickReplyDinamico"):
        button_message(recipient_id, contenido, msj["Respuesta"]["options"],access_token)
        return entidad, ans, estado, tipo, msj["Respuesta"]["options"]
        
    elif(tipo=="input" or tipo=="inputDinamico"):
        text_message(recipient_id, contenido,access_token)
        return entidad, ans, estado, tipo, 'vacio'
    
    elif(tipo=="informativo" or tipo=="informativoDinamico"):
        text_message(recipient_id, contenido,access_token)
        return entidad, str('next'), estado, tipo, 'vacio'    
    elif(tipo=="slide" or tipo=="slideBuy" or tipo=="slideDinamico"):
        slide_message(recipient_id, contenido,access_token)
        return entidad, str('next'), estado, tipo, 'vacio'
    elif(tipo=="ticket"):
        ticket_message(recipient_id, contenido,access_token)
        return entidad, str('next'), estado, tipo, 'vacio'
    else:
        return entidad, str('hey'), estado, tipo, 'vacio'

In [11]:
class Nucleo_Cliente2(threading.Thread):
    def on_response(self, ch, method, props, body):
        if self.corr_id == props.correlation_id:
            self.response = body
            
    def __init__(self, nombreHilo):    
        threading.Thread.__init__(self, name=nombreHilo, target=Nucleo_Cliente2.run)
        self.nombrehilo=nombreHilo
        self.connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
        self.channel = self.connection.channel()
        self.result = self.channel.queue_declare(queue='',exclusive=True)
        self.queue_name = self.result.method.queue
        self.corr_id = ' ' 
        
    def run(self):
        try:                
            self.response = None
            self.response = self.call()
            print(" [.] Got %r", self.response) 
        except Exception as error:
            print("Error run:",str(error))            
    def call(self):
        print("hilo consume")
        cont = 0        
        while(cont>=0):
            self.response = None
            while self.response is None:
                self.connection.process_data_events()            
            if len(self.response)>10:
                data = json.loads(self.response)
                print('\n_________________________________________________________________________________________________________________________')
                print('Respuesta del Nucleo Logico-> ',data)
                entidad, resp, estado, tipo, opciones= conv(data)

In [12]:

#-------------Creación de la REST API con Flask-------------
app = Flask(__name__)
#---------------PASO 1. Recepción de mensajes enviados por la API de Facebook.--------------------------
@app.route("/apiREST/v1", methods=['GET', 'POST'])
def receive_message():
    peticiones = 0
    #---------------PASO 2. Inicio: Identificar el tipo de petición que se está solicitando.--------------------------
    if request.method == 'GET':
        print("metodo GET...")
        token_sent = request.args.get("hub.verify_token")
        #---------------PASO 2.1. Llamada a la función que valida la cadena de verificación y se retorna a la petición GET la validación de la función.--------------------------
        return verify_fb_token(token_sent,request.args.get("hub.challenge"))
    else:
        #---------------PASO 2.2. Recepción de mensajes de una operacion tipo POST.--------------------------
        nombreHilo="Hilo_"+str(peticiones)
        N_C = Nucleo_Cliente2(nombreHilo)
        print(request)
        content = request.get_json(force=True)
        output = request.get_json()
        print('\n_________________________________________________________________________________________________________________________')
        print("Mesaje de la API de Facebook-> ",output)
        #return "Message Processed"  
        for event in output['entry']:
            messaging = event['messaging']
            id_face = event['id']
            for message in messaging:
                if message.get('message') or message.get('postback'):
                    recipient_id = message['sender']['id']                 
                    enviarNucleo(recipient_id, id_face, messaging,N_C)  
                    #envio de mensaje proveniente de fb a nucleo lógico del sistema
    
    return "Message Processed"        
#-------------Iniciando la REST API-------------
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Apr/2020 16:47:54] "POST /apiREST/v1 HTTP/1.1" 200 -


<Request 'http://7167a37c.ngrok.io/apiREST/v1' [POST]>

_________________________________________________________________________________________________________________________
Mesaje de la API de Facebook->  {'object': 'page', 'entry': [{'id': '2153123008120093', 'time': 1585781273822, 'messaging': [{'sender': {'id': '2374218029278335'}, 'recipient': {'id': '2153123008120093'}, 'timestamp': 1585781096584, 'message': {'mid': 'm_5mZEyJ7R4TLRzhvhAPAw5DAiVxFEtHfcdpDof8LBiHlN-N9C6X7Ke_GB6f27QouXD_OteelS30OQ2Zhh9cicCQ', 'text': 'Ok'}}]}]}
hilo consume

_________________________________________________________________________________________________________________________
Mensaje enviado al Nucleo Logico->  {"payload": {"robot": "none", "id_face": "2153123008120093", "recipient_id": "2374218029278335", "access_token": "none", "content": "Ok", "type_resp": "message", "state": "none", "blocktype": "none", "opciones": "none", "entitites": " ", "entities": {}}}

_______________________________

127.0.0.1 - - [01/Apr/2020 16:47:55] "POST /apiREST/v1 HTTP/1.1" 200 -


<Request 'http://7167a37c.ngrok.io/apiREST/v1' [POST]>

_________________________________________________________________________________________________________________________
Mesaje de la API de Facebook->  {'object': 'page', 'entry': [{'id': '2153123008120093', 'time': 1585781274822, 'messaging': [{'sender': {'id': '2374218029278335'}, 'recipient': {'id': '2153123008120093'}, 'timestamp': 1585781198548, 'message': {'mid': 'm_0c2Z-5hmAObr3ze_VwWkGDAiVxFEtHfcdpDof8LBiHnOl7r8wdNoKms6EWHcdZv4yaju_UFk3yGuk5yyHUgY6g', 'text': 'Ok'}}]}]}
hilo consume
_________________________________________________________________________________________________________________________

Mensaje enviado al Nucleo Logico->  {"payload": {"robot": "none", "id_face": "2153123008120093", "recipient_id": "2374218029278335", "access_token": "none", "content": "Ok", "type_resp": "message", "state": "none", "blocktype": "none", "opciones": "none", "entitites": " ", "entities": {}}}

_______________________________

...

hilo consume
_________________________________________________________________________________________________________________________
Mensaje enviado al Nucleo Logico->  {"payload": {"robot": "none", "id_face": "2153123008120093", "recipient_id": "2374218029278335", "access_token": "none", "content": "Jwrurwut", "type_resp": "message", "state": "none", "blocktype": "none", "opciones": "none", "entitites": " ", "entities": {}}}


_________________________________________________________________________________________________________________________
Respuesta del Nucleo Logico->  {'Respuesta': {'content': 'Hola, espero que estes teniendo un excelente dia.', 'next_id': 'info_bot2', 'blockType': 'informativo', 'contentType': 'tex', 'typingTime': '1', 'payload': {'content': '', 'state': 'Saludo2', 'recipient_id': '2374218029278335', 'entities': {'prueba_var': 'Jwrurwut'}, 'access_token': 'EAAHNkscg2TcBAPwb5WSaxOM7IjDf57ZAoYu5TwDdUckaTK7dJc6nWq3fCaZBcKZBEZB3jHZCvbykRpYZAzpUHfIwmZCa45OcnXzN

127.0.0.1 - - [01/Apr/2020 16:49:01] "POST /apiREST/v1 HTTP/1.1" 200 -
127.0.0.1 - - [01/Apr/2020 16:49:01] "POST /apiREST/v1 HTTP/1.1" 200 -


<Request 'http://7167a37c.ngrok.io/apiREST/v1' [POST]>

_________________________________________________________________________________________________________________________
Mesaje de la API de Facebook->  {'object': 'page', 'entry': [{'id': '2153123008120093', 'time': 1585781330109, 'messaging': [{'sender': {'id': '2374218029278335'}, 'recipient': {'id': '2153123008120093'}, 'timestamp': 1585781330179, 'delivery': {'mids': ['m_6ySHwB7RQRu78KHLufM2XzAiVxFEtHfcdpDof8LBiHnH87Y8CwlxnN4d6ZsaH6Ivzo2QIkMbU8htWCdYqOCFHQ', 'm_Q0AJ8HKtMvCrIg-c2EOrIDAiVxFEtHfcdpDof8LBiHlZThYH7gprwAkBypVlnWMmHJVK-Y3MFN-o85EFL7lwOA', 'm_Dc09wmu_JwCpFQ6m6mkwRjAiVxFEtHfcdpDof8LBiHmSYPW8LY4zFpkJOWdOaLgR085_K_k6BBl8wELER5yOaQ'], 'watermark': 1585781329662}}]}]}
<Request 'http://7167a37c.ngrok.io/apiREST/v1' [POST]>

_________________________________________________________________________________________________________________________
Mesaje de la API de Facebook->  {'object': 'page', 'entry': [{'id': '2153123008

127.0.0.1 - - [01/Apr/2020 16:53:44] "POST /apiREST/v1 HTTP/1.1" 200 -


<Request 'http://7167a37c.ngrok.io/apiREST/v1' [POST]>

_________________________________________________________________________________________________________________________
Mesaje de la API de Facebook->  {'object': 'page', 'entry': [{'id': '2153123008120093', 'time': 1585781621860, 'messaging': [{'sender': {'id': '2374218029278335'}, 'recipient': {'id': '2153123008120093'}, 'timestamp': 1585781621917, 'delivery': {'mids': ['m_6ySHwB7RQRu78KHLufM2XzAiVxFEtHfcdpDof8LBiHnH87Y8CwlxnN4d6ZsaH6Ivzo2QIkMbU8htWCdYqOCFHQ', 'm_Q0AJ8HKtMvCrIg-c2EOrIDAiVxFEtHfcdpDof8LBiHlZThYH7gprwAkBypVlnWMmHJVK-Y3MFN-o85EFL7lwOA', 'm_Dc09wmu_JwCpFQ6m6mkwRjAiVxFEtHfcdpDof8LBiHmSYPW8LY4zFpkJOWdOaLgR085_K_k6BBl8wELER5yOaQ'], 'watermark': 1585781329662}}]}]}
